## Summary of Study Results

With the data downloads from Replica, create a single table with a row for each origin station and columns for “car trips to areas around other destination stations on the other lines” and another column for “train trips”

In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

In [2]:
import replica_utils

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
gcs_path = "calitp-analytics-data/data-analyses/big_data/MetroLink_LinkUS/"

In [14]:
analysis_result_names = [ "line_origin_union_station",
                         "riversideline_origin_montebello", "riversideline_origin_industry", "riversideline_origin_pomona",  
                         "riversideline_origin_ontario", "riversideline_origin_pedley",  "riversideline_origin_riverside",
                         
                         "sbline_origin_calstate_la", "sbline_origin_el_monte",  "sbline_origin_baldwin_park",
                         "sbline_origin_covina",  "sbline_origin_pomona_north", "sbline_origin_fontana",
                         "sbline_origin_claremont", "sbline_origin_montclair", "sbline_origin_upland", "sbline_origin_rancho_cucamonga", 
                         "sbline_origin_rialto", "sbline_origin_san_bernardino_depot", "sbline_origin_san_bernardino_dt" 
                        ]

In [15]:
# analysis_result_names_subset_test = ["sbline_origin_covina", "sbline_origin_calstate_la", 
#                                 "riversideline_origin_industry",  "riversideline_origin_montebello"]

In [16]:
results = []
for analysis in analysis_result_names:
    with get_fs().open(f"{gcs_path}replica_data_downloads/replica-socal_travel_analysis_{analysis}-trips_dataset.csv") as f:
        df = to_snakecase(pd.read_csv(f))
        
        station = analysis.split('_origin_')[1]  # Get the part after '_origin_'
        station_name = station.replace('_', ' ').upper()
        
        line = analysis.split('_origin_')[0]
        
        metrolink_line = replica_utils.define_transit_line(line) 
        n_total_trips = len(df)
        n_private_auto_trips = len(df[df.primary_mode=="private_auto"])
        pct_private_auto_trips = ((len(df[df.primary_mode=="private_auto"]))/(len(df)))
        n_public_transit_trips = (len(df[df.primary_mode=="public_transit"]))
        pct_public_transit_trips = ((len(df[df.primary_mode=="public_transit"]))/(len(df)))

        auto_mean_min, auto_median_min, auto_mean_miles, auto_median_miles = replica_utils.calc_auto_travel_info(df)
        transit_mean_min, transit_median_min, transit_mean_miles, transit_median_miles = replica_utils.calc_transit_travel_info(df)

        ## set up the table for all the results
        results.append({
                'station_name': station_name,
                'metrolink_line': metrolink_line,
                'total_trips': n_total_trips,
                'n_auto_trips_to_other_station_areas': n_private_auto_trips,
                'pct_auto_trips_to_other_station_areas':pct_private_auto_trips,
                'n_tranist_trips_to_other_station_areas': n_public_transit_trips,
                'pct_transit_trips_to_other_station_areas': pct_public_transit_trips,
                'auto_mean_minutes': auto_mean_min,
                'auto_median_minutes': auto_median_min,
                'auto_mean_miles': auto_mean_miles,
                'auto_median_miles': auto_median_miles,
                'transit_mean_minutes': transit_mean_min,
                'transit_median_minutes': transit_median_min,
                'transit_mean_miles': transit_mean_miles,
                'transit_median_miles': transit_median_miles,
                })
        # results = results.astype({
        #         "total_trips": "int",
        #         "n_auto_trips_to_other_station_areas": "int",
        #         "pct_auto_trips_to_other_station_areas": "int",
        #         "n_tranist_trips_to_other_station_areas": "int"
        #     })


        
result_summary = pd.DataFrame(results)
        

/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = to_snakecase(pd.read_csv(f))
/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = to_snakecase(pd.read_csv(f))
/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = to_snakecase(pd.read_csv(f))
/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = to_snakecase(pd.read_csv(f))
/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = to_snakecase(pd.read_csv(f))
/tmp/ipykernel_643/1017489968.py:4: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory

In [17]:
result_summary

,station_name,metrolink_line,total_trips,n_auto_trips_to_other_station_areas,pct_auto_trips_to_other_station_areas,n_tranist_trips_to_other_station_areas,pct_transit_trips_to_other_station_areas,auto_mean_minutes,auto_median_minutes,auto_mean_miles,auto_median_miles,transit_mean_minutes,transit_median_minutes,transit_mean_miles,transit_median_miles
0,UNION STATION,Both,18657,10339,0.554162,938,0.050276,31.680143,24.0,21.165722,14.3,78.638593,64.0,21.760768,21.10
1,MONTEBELLO,Riverside Line,42917,22560,0.525666,655,0.015262,13.369238,7.0,7.194459,2.7,42.964885,30.0,7.341221,3.50
2,INDUSTRY,Riverside Line,8588,5436,0.632976,51,0.005939,26.223142,23.0,17.304213,13.9,97.647059,82.0,26.864706,23.90
3,POMONA,Riverside Line,19454,10982,0.564511,186,0.009561,15.845110,11.0,9.885230,5.4,58.634409,47.0,13.086559,10.60
4,ONTARIO,Riverside Line,63621,35456,0.557300,74,0.001163,18.636084,16.0,11.136967,9.1,84.351351,82.5,17.785135,16.00
5,PEDLEY,Riverside Line,8523,5720,0.671125,6,0.000704,21.623776,19.0,12.747238,9.8,89.500000,68.0,26.750000,19.70
6,RIVERSIDE,Riverside Line,31087,19783,0.636375,236,0.007592,13.263762,11.0,8.226235,6.8,47.627119,34.5,7.964831,6.50
7,CALSTATE LA,San Bernardino Line,9447,6282,0.664973,186,0.019689,20.878860,15.0,12.178383,6.7,62.935484,46.0,12.464516,7.45
8,EL MONTE,San Bernardino Line,9072,6596,0.727072,162,0.017857,24.060794,19.0,15.411810,10.7,69.302469,60.0,16.438272,14.30
9,BALDWIN PARK,San Bernardino Line,10525,7330,0.696437,179,0.017007,21.493179,18.0,14.157817,12.0,77.217877,72.0,19.465363,18.00


In [ ]:
# df_auto.columns

In [ ]:
# df.primary_mode.value_counts()